In [1]:
import numpy as np
import functools as ft
import networkx as nx

In [2]:
def hdv(d):
    return np.random.choice([-1, 1], d)


def zero(d):
    return np.zeros((d,))


def bind(xs):
    return ft.reduce(lambda x, y: x * y, xs)


def bundle(xs):
    return ft.reduce(lambda x, y: x + y, xs)


def p(d):
    return np.random.shuffle(np.eye(d))


def inverse(P):
    return np.linalg.inv(P)


def permute(P, H):
    return P.dot(H)


def cosine_similarity(A, B):
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        return 0

    return np.dot(A, B) / (norm_A * norm_B)


def similarity(A, B):
    return np.dot(A, B) / len(A)


sim = cosine_similarity

In [3]:
class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def getVector(self, label):
        return self.vectors[label]

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: sim(V, x[1]))

In [4]:
def initVertices(graph, d=10000):
    for n in graph:
        graph.nodes[n]["hdv"] = hdv(d)

In [5]:
def initNodeMem(graph, memory):
    for n in graph:
        xs = map(lambda x: graph.nodes[x]["hdv"], list(graph.neighbors(n)))
        graph.nodes[n]["mem"] = bundle(xs)
        memory.addVector(f"mem{n}", graph.nodes[n]["mem"])

In [6]:
def retrain(graph, memory, threshold, iter=15):
    count = 0
    for i in range(iter):
        for n in graph:
            mem = graph.nodes[n]["mem"]
            finish = True
            for neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.neighbors(graph, n))
            ):
                if sim(mem, neighbor) < threshold:
                    mem = bundle([mem, neighbor])
                    finish = False
                    print("here")

            for non_neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.non_neighbors(graph, n))
            ):
                if sim(mem, non_neighbor) > threshold:
                    mem = bundle([mem, -non_neighbor])
                    finish = False
                    print("here")

            if finish:
                return

            graph.nodes[n]["mem"] = mem
            memory.addVector(f"mem{n}_{i}", mem)

In [7]:
def initGraph(graph, memory):
    G = []
    for n in graph.nodes():
        G.append(bind([graph.nodes[n]["hdv"], graph.nodes[n]["mem"]]))
    G = bundle(G) / 2
    memory.addVector("G", G)
    return G

In [8]:
def checkEdge(G, A, B, threshold):
    return sim(B, bind([G, A])) > threshold

In [9]:
def nodeMemoryReconstruction(G, xs, iter=15):
    if iter == 0:
        return list(map(lambda H: bind([G, H]), xs))

    mems = nodeMemoryReconstruction(G, xs, iter - 1)

    newMems = []
    b = [bind(x) for x in zip(mems, xs)]
    for i in range(len(xs)):
        bmem = bundle(b[:i] + b[i + 1 :])
        newMems.append(bind([xs[i], bundle([G, -bmem])]))

    return newMems

In [10]:
def graphMemoryReconstruction(G, xs, iter=15):
    raise NotImplementedError
    if iter == 0:
        return (G, zeros(len(G)))

    (Gi, Ni) = graphMemoryReconstruction(G, xs, iter - 1)
    Gii = bundle([Gi, -Ni])

    for i in range(iter):
        print(i)

    return bundle([Gi, -edges])

In [11]:
# visited = dict()
visited = {}


def shortestPath(G, A, B, xs, distance, threshold, memory):
    label, value = memory.cleanup(A)

    if np.array_equiv(A, B):
        return 0, label

    if label in visited and visited[label][0] <= distance:
        return visited[label][1], visited[label][2]

    visited[label] = [distance, 99, "memnill"]

    neighbours = list(filter(lambda x: checkEdge(G, A, x, threshold), xs))

    if len(neighbours) == 0:
        return 99, "memnill"

    dis, lab = min(
        list(
            map(
                lambda a: shortestPath(G, a, B, xs, distance + 1, threshold, memory),
                neighbours,
            )
        ),
        key=lambda x: x[0],
    )
    visited[label][1] = dis + 1
    visited[label][2] = label + lab
    return dis + 1, label + lab

---


# Tests


In [12]:
def testCheckEdge(G, graph, threshold):
    count = 0

    for n in graph.nodes():
        for m in graph.nodes():
            if n == m:
                continue
            exist = graph.has_edge(n, m)
            check = checkEdge(
                G, graph.nodes[n]["hdv"], graph.nodes[m]["hdv"], threshold
            )
            if exist != check:
                count += 1
                print(n, m, exist, check)

    print(count, "%.5f" % round(count / EDGES, 5))

In [13]:
def testNodeMemoryReconstruction(G, graph, iter=15):
    for i in range(iter):
        memsi = nodeMemoryReconstruction(
            G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=i
        )
        print(
            f"{0}_{i:02} =>",
            "%.10f" % abs(sim(graph.nodes[0]["mem"], memsi[0])),
        )

---


# Main


In [14]:
NODES, EDGES = 30, 150
DIMENSIONS, THRESHOLD, ITER = 10000, 0.047, 4

In [15]:
def main():
    graph = nx.gnm_random_graph(NODES, EDGES)
    memory = ItemMemory()

    initVertices(graph, DIMENSIONS)
    initNodeMem(graph, memory)
    retrain(graph, memory, THRESHOLD, ITER)
    G = initGraph(graph, memory)
    print("G =>", G)
    # testNodeMemoryReconstruction(G, graph, 1)
    # testCheckEdge(G, graph, THRESHOLD)


# main()

In [16]:
# graph = nx.gnm_random_graph(NODES, EDGES)
# memory = ItemMemory()

# initVertices(graph, DIMENSIONS)
# initNodeMem(graph, memory)
# G = initGraph(graph, memory)
# print("G =>", G)

In [17]:
# testNodeMemoryReconstruction(G, graph, 15)

In [18]:
# sim(graph.nodes[0]["mem"], graph.nodes[0]["mem"])

---


In [19]:
mem = ItemMemory()

g = nx.gnm_random_graph(NODES, 45)
initVertices(g, DIMENSIONS)
initNodeMem(g, mem)
G = initGraph(g, mem)

for n in g.nodes():
    print(g.edges(n))

[(0, 10), (0, 17), (0, 21)]
[(1, 20)]
[(2, 10), (2, 20)]
[(3, 6), (3, 9), (3, 7), (3, 14), (3, 13)]
[(4, 28), (4, 17), (4, 29)]
[(5, 7), (5, 21), (5, 14)]
[(6, 3), (6, 9), (6, 25)]
[(7, 5), (7, 23), (7, 12), (7, 10), (7, 18), (7, 3)]
[(8, 15), (8, 20)]
[(9, 16), (9, 3), (9, 6), (9, 26)]
[(10, 0), (10, 2), (10, 7), (10, 14)]
[(11, 20), (11, 21)]
[(12, 7), (12, 17), (12, 21), (12, 19)]
[(13, 3)]
[(14, 18), (14, 23), (14, 5), (14, 3), (14, 10)]
[(15, 8)]
[(16, 22), (16, 9), (16, 19)]
[(17, 0), (17, 12), (17, 4)]
[(18, 14), (18, 7), (18, 24)]
[(19, 16), (19, 12), (19, 20)]
[(20, 1), (20, 28), (20, 11), (20, 2), (20, 27), (20, 8), (20, 19), (20, 22)]
[(21, 27), (21, 5), (21, 12), (21, 11), (21, 0)]
[(22, 16), (22, 20)]
[(23, 7), (23, 14), (23, 25)]
[(24, 18), (24, 25)]
[(25, 23), (25, 24), (25, 6)]
[(26, 9)]
[(27, 21), (27, 20)]
[(28, 20), (28, 4)]
[(29, 4)]


In [20]:
hdvs = list(map(lambda x: g.nodes[x]["hdv"], g.nodes()))
origin = 0

In [21]:
for n in g.nodes():
    print(
        f"{n:02} =>",
        "T" if mem.cleanup(hdvs[n])[0] == f"mem{n}" else "F",
        mem.cleanup(hdvs[n])[0],
    )

00 => F mem17
01 => F mem20
02 => F mem10
03 => F mem13
04 => F mem29
05 => F mem21
06 => F mem25
07 => F mem23
08 => F mem15
09 => F mem26
10 => F mem2
11 => F mem21
12 => F mem17
13 => F mem3
14 => F mem5
15 => F mem8
16 => F mem22
17 => F mem0
18 => F mem24
19 => F mem16
20 => F mem1
21 => F mem11
22 => F mem16
23 => F mem25
24 => F mem25
25 => F mem24
26 => F mem9
27 => F mem21
28 => F mem4
29 => F mem4


In [22]:
for m in g.nodes():
    if origin == m:
        continue
    path = nx.shortest_path(g, origin, m)
    oracle = len(path) - 1
    visited = {}
    test = shortestPath(G, hdvs[origin], hdvs[m], hdvs, 0, THRESHOLD, mem)
    testPath = list(map(int, test[1].split("mem")[1:]))
    print(
        f"{origin}_{m:02} =>",
        oracle,
        test[0],
        "T  " if oracle == test[0] else "F X",
        path,
        testPath,
        nx.is_path(g, testPath),
    )

0_01 => 4 4 T   [0, 10, 2, 20, 1] [17, 2, 10, 1, 20] False
0_02 => 2 2 T   [0, 10, 2] [17, 2, 10] False
0_03 => 3 3 T   [0, 10, 7, 3] [17, 2, 23, 13] False
0_04 => 2 2 T   [0, 17, 4] [17, 0, 29] False


0_05 => 2 2 T   [0, 21, 5] [17, 11, 21] False
0_06 => 4 4 T   [0, 10, 7, 3, 6] [17, 2, 23, 13, 25] False
0_07 => 2 2 T   [0, 10, 7] [17, 2, 23] False
0_08 => 4 4 T   [0, 10, 2, 20, 8] [17, 2, 10, 1, 15] False
0_09 => 4 4 T   [0, 10, 7, 3, 9] [17, 2, 23, 13, 26] False
0_10 => 1 1 T   [0, 10] [17, 2] False
0_11 => 2 2 T   [0, 21, 11] [17, 11, 21] False
0_12 => 2 2 T   [0, 17, 12] [17, 0, 17] True
0_13 => 4 4 T   [0, 10, 7, 3, 13] [17, 2, 23, 13, 3] False
0_14 => 2 2 T   [0, 10, 14] [17, 2, 5] False
0_15 => 5 5 T   [0, 10, 2, 20, 8, 15] [17, 2, 10, 1, 15, 8] False
0_16 => 4 5 F X [0, 17, 12, 19, 16] [17, 2, 10, 1, 16, 22] False
0_17 => 1 1 T   [0, 17] [17, 0] True
0_18 => 3 3 T   [0, 10, 7, 18] [17, 2, 23, 24] False
0_19 => 3 4 F X [0, 17, 12, 19] [17, 2, 10, 1, 16] False
0_20 => 3 3 T   [0, 10, 2, 20] [17, 2, 10, 1] False
0_21 => 1 1 T   [0, 21] [17, 11] False
0_22 => 4 4 T   [0, 10, 2, 20, 22] [17, 2, 10, 1, 16] False
0_23 => 3 3 T   [0, 10, 7, 23] [17, 2, 23, 25] False
0_24 => 4 4 T   

In [23]:
testCheckEdge(G, g, THRESHOLD)

0 0.00000
